In [43]:
import pandas as pd
import numpy as np

!wget -q https://raw.githubusercontent.com/crtez/fantasy_reference/refs/heads/main/stats.csv

# Read the CSV file
df = pd.read_csv('stats.csv')

# Convert numeric columns to appropriate types
numeric_cols = ['hit', 'strikeout', 'walk', 'p_out', 'p_win', 'p_earned_run', 
                'p_save', 'p_balk', 'p_quality_start', 'p_hit_by_pitch', 
                'p_hold', 'p_total_pickoff']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate points using the old scoring system
df['old_points'] = (
    df['p_win'] * 8 +
    df['p_save'] * 8 +
    df['p_out'] * 1 +
    df['hit'] * -1.3 +
    df['p_earned_run'] * -3 +
    df['walk'] * -1.3 +
    df['p_hit_by_pitch'] * -1.3 +
    df['strikeout'] * 3
)

# Calculate points using the new scoring system (old + additional stats)
df['new_points'] = (
    df['old_points'] +
    df['p_balk'] * -1 +
    df['p_hold'] * 4 +
    df['p_total_pickoff'] * 1 +
    df['p_quality_start'] * 3 +
    df['p_save'] * 5 + # 5 more points for saves
    df['p_blown_save'] * -5 # minus points for blown saves
)

# Add rank columns
df['old_rank'] = df['old_points'].rank(ascending=False, method='min').astype(int)
df['new_rank'] = df['new_points'].rank(ascending=False, method='min').astype(int)
df['rank_improvement'] = df['old_rank'] - df['new_rank']

# Display the top 20 players with their ranks
top_players = df[['last_name, first_name', 'p_save', 'old_points', 'new_points', 'old_rank', 'new_rank']].rename(columns={'p_save': 'saves_last_season'})
top_players = top_players.sort_values(by='new_points', ascending=False)

print("Top 100 players by new scoring system with ranks:")
with pd.option_context('display.max_rows', None):
    display(top_players.head(100))

# List of the 20 specific players you provided
specific_players = [
    "Helsley, Ryan", "Clase, Emmanuel", "Finnegan, Kyle", "Suarez, Robert", 
    "Hader, Josh", "Iglesias, Raisel", "Yates, Kirby", "Holmes, Clay", 
    "Foley, Jason", "Díaz, Alexis", "Miller, Mason", "Jansen, Kenley", 
    "Estévez, Carlos", "Doval, Camilo", "Bednar, David", "Duran, Jhoan", 
    "Kimbrel, Craig", "Fairbanks, Pete", "Scott, Tanner", "Muñoz, Andrés"
]

# Filter the dataframe to only include these players
specific_df = df[df['last_name, first_name'].isin(specific_players)]

# Create a dataframe showing rank improvements for these specific players
specific_rank_changes = specific_df[['last_name, first_name', 'p_save', 'old_points', 'new_points', 'old_rank', 'new_rank', 'rank_improvement']].rename(columns={'p_save': 'saves_last_season'})
specific_rank_changes = specific_rank_changes.sort_values(by='rank_improvement', ascending=False)

print("\nRank improvements for the 20 players with the most saves last season:")
display(specific_rank_changes)

Top 100 players by new scoring system with ranks:


,"last_name, first_name",saves_last_season,old_points,new_points,old_rank,new_rank
209,"Skubal, Tarik",0,1009.2,1075.2,1,1
520,"Wheeler, Zack",0,970.3,1049.3,2,2
79,"Sale, Chris",0,966.6,1021.6,3,3
778,"Helsley, Ryan",49,744.1,969.1,15,4
152,"Clase, Emmanuel",47,747.7,967.7,14,5
695,"Gilbert, Logan",0,887.3,954.3,4,6
478,"Cease, Dylan",0,867.8,915.8,5,7
252,"Ragans, Cole",0,836.3,900.3,6,8
171,"Lugo, Seth",0,779.8,845.8,7,9
229,"Burnes, Corbin",0,777.5,842.5,9,10



Rank improvements for the 20 players with the most saves last season:


,"last_name, first_name",saves_last_season,old_points,new_points,old_rank,new_rank,rank_improvement
629,"Foley, Jason",28,408.1,559.1,118,77,41
2,"Duran, Jhoan",23,437.3,569.3,102,71,31
345,"Finnegan, Kyle",38,510.5,675.5,75,44,31
758,"Jansen, Kenley",27,461.3,574.3,95,68,27
558,"Bednar, David",23,333.5,425.5,161,135,26
582,"Fairbanks, Pete",23,353.1,452.1,150,124,26
734,"Díaz, Alexis",28,396.3,517.3,124,100,24
571,"Scott, Tanner",22,567.4,711.4,57,35,22
742,"Suarez, Robert",36,583.6,737.6,55,33,22
457,"Yates, Kirby",33,667.1,833.1,31,12,19
